# MARKOV MODEL CLASSIFIER

Dado 2 documentos com poesias, classificar de qual autor é cada linha da poesia

In [1]:
# FILES:
#!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
#!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
    'edgar_allan_poe.txt',
    'robert_frost.txt'
]

In [9]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
    print(f'{f} corresponds to label {label}')

    for line in open(f):
        line = line.rstrip().lower()
        if line:
            # remove punctuation
            line = line.translate(str.maketrans('', '', string.punctuation))

            input_texts.append(line)
            labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [12]:
len(input_texts), len(labels)

(2158, 2158)

In [13]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [14]:
len(Ytrain), len(Ytest)

(1618, 540)

In [15]:
train_text[:5]

['two roads diverged in a wood and i',
 'but when im gone of course i cant stay here',
 'as often as he had in the tail of the night',
 'to the loved object  so the tear to the lid',
 'and come and make your summer dwelling here']

In [16]:
Ytrain[:5]

[1, 1, 1, 0, 1]

In [17]:
idx = 1
word2idx = {'<unk>': 0}

In [18]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
        if token not in word2idx:
            word2idx[token] = idx
            idx += 1

In [19]:
word2idx

{'<unk>': 0,
 'two': 1,
 'roads': 2,
 'diverged': 3,
 'in': 4,
 'a': 5,
 'wood': 6,
 'and': 7,
 'i': 8,
 'but': 9,
 'when': 10,
 'im': 11,
 'gone': 12,
 'of': 13,
 'course': 14,
 'cant': 15,
 'stay': 16,
 'here': 17,
 'as': 18,
 'often': 19,
 'he': 20,
 'had': 21,
 'the': 22,
 'tail': 23,
 'night': 24,
 'to': 25,
 'loved': 26,
 'object': 27,
 'so': 28,
 'tear': 29,
 'lid': 30,
 'come': 31,
 'make': 32,
 'your': 33,
 'summer': 34,
 'dwelling': 35,
 'sacred': 36,
 'sun': 37,
 'all': 38,
 'who': 39,
 'weeping': 40,
 'bless': 41,
 'thee': 42,
 'one': 43,
 'thing': 44,
 'you': 45,
 'help': 46,
 'liking': 47,
 'about': 48,
 'john': 49,
 'was': 50,
 'something': 51,
 'brushed': 52,
 'across': 53,
 'my': 54,
 'mind': 55,
 'passed': 56,
 'place': 57,
 'how': 58,
 'daring': 59,
 'an': 60,
 'ambition': 61,
 'yet': 62,
 'deep': 63,
 'terror': 64,
 'she': 65,
 'spoke': 66,
 'letting': 67,
 'sink': 68,
 'her': 69,
 'our': 70,
 'thoughts': 71,
 'they': 72,
 'were': 73,
 'palsied': 74,
 'sere': 75,
 '

In [20]:
len(word2idx)

2482

In [22]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
    tokens = text.split()
    line_as_int = [word2idx[token] for token in tokens]
    train_text_int.append(line_as_int)

for text in test_text:
    tokens = text.split()
    # Different: there will be words in test that are not present in train
    line_as_int = [word2idx.get(token, 0) for token in tokens]
    test_text_int.append(line_as_int)

In [23]:
train_text_int[:5]

[[1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11, 12, 13, 14, 8, 15, 16, 17],
 [18, 19, 18, 20, 21, 4, 22, 23, 13, 22, 24],
 [25, 22, 26, 27, 28, 22, 29, 25, 22, 30],
 [7, 31, 7, 32, 33, 34, 35, 17]]

In [24]:
test_text_int[:5]

[[25, 31, 25, 152, 459, 10, 720, 12, 25, 1610],
 [22, 930, 1726, 73, 0, 172, 82, 0],
 [22, 0, 178, 69, 0, 1013],
 [25, 0, 119, 25, 534, 45, 378, 121, 22, 0],
 [13, 1370, 0, 329, 718, 330, 172]]

In [25]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [26]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [29]:
np.unique(A1)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 17., 18., 26., 40.])

In [30]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [33]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [34]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3318912237330037, 0.6681087762669963)

In [35]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [36]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [37]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9962917181705809


In [38]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8037037037037037


In [39]:
from sklearn.metrics import confusion_matrix, f1_score

In [40]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 531,    6],
       [   0, 1081]])

In [41]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 89,  96],
       [ 10, 345]])

In [42]:
f1_score(Ytrain, Ptrain)

0.9972324723247232

In [43]:
f1_score(Ytest, Ptest)

0.8668341708542714